<a href="https://colab.research.google.com/github/JamesLabUofT/GEE_Workshop/blob/main/scripts/working_with_awesome_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting and Processing data from Awesome GEE Community Catalog


## Module Overview

This module introduces participants to accessing, analyzing, and visualizing forest-related geospatial datasets from the Awesome GEE Community Data Catalog using Google Earth Engine (GEE) and Python. Through hands-on exercises in Google Colab, participants will explore forest disturbance, land cover, and species composition across Canada using high-resolution datasets.

## Learning Objectives:
By the end of this module, participants will be able to:

* Understand the structure and content of selected forest datasets in the Awesome GEE Catalog
* Use Python and the Earth Engine API to access, filter, and visualize geospatial data
* Work with both raster (ImageCollections) and vector (FeatureCollections) datasets
* Perform spatial and temporal analysis on forest disturbance and land cover
Export processed data for further analysis or reporting

## Topics Covered:
**Introduction to the Awesome GEE Community Catalog**

* Overview of the catalog and its purpose
* Navigating the catalog and locating datasets

**Working with FeatureCollections: NBAC Dataset
Dataset: National Burned Area Composite (NBAC)**

* Understanding vector data in GEE
* Loading and filtering NBAC by year and region
* Visualizing burned areas on interactive maps
* Calculating burned area statistics
* Exporting filtered data to CSV or Google Drive

**Using the NBAC perimeters, lets explore some forest cover**

**Exploring Raster Datasets**

a. High-Resolution Annual Forest Land Cover (1984–2022)
* Getting annual forest cover per class prior to fire
* Detecting forest cover change over time - how did the forest change in the from 2000-2022


b. Canada Long-Term Tree Species (1984–2022)
* Mapping tree species
* Can we get the percentage of black spruce within each fire?


c. Canada Landsat-Derived Forest Harvest Disturbance (1985–2020)
* Mapping harvest disturbances - were there any harvest disturbances detected within fire perimeters between 1985 and 2020?
* Comparing harvest vs. fire impacts (not covered today)
* Temporal analysis of disturbance trends (not covered today)



**Data Extraction and Visualization**

* Reducing data over custom regions (e.g., provinces, ecozones)
* Exporting time-series data to Pandas DataFrames
* Creating charts and maps using matplotlib and geemap
* Exporting results for use in reports or further analysis



## Introduction to the Awesome GEE Community Catalog


✅ Expand access to valuable datasets not included in the official GEE catalog
🌍 Support reproducible science by sharing preprocessed, ready-to-use data
🧑‍🤝‍🧑 Foster collaboration across the global geospatial community
📈 Accelerate research in climate, ecology, forestry, urban studies, and more

https://gee-community-catalog.org/

**environment setup**

In [5]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipyleaflet

Authenticate and intialize ee

In [7]:
from google.colab import userdata
ee.Authenticate()
id = userdata.get('user_id')
ee.Initialize(project=id) # project='ee-userid'

# Working with Feature Collections: NBAC

In this section, participants will explore how to work with vector data in Google Earth Engine using the NBAC dataset, which maps burned areas across Canada. This hands-on module introduces the concept of FeatureCollections, a core data structure in GEE used to represent geographic features like points, lines, and polygons.

FeatureCollections are the vector equivalent of ImageCollections in Earth Engine. They store geographic features—such as fire perimeters, forest plots, or administrative boundaries—along with associated attributes (e.g., year, area, cause). In this session, we’ll use the National Burned Area Composite (NBAC) to:

* Load and explore a FeatureCollection
* Filter features by year and region
* Visualize fire perimeters on an interactive map
* Calculate statistics like total burned area
* Export selected features for further analysis

We'll start with loading in the NBAC from https://gee-community-catalog.org/projects/nbac/#citation

In [8]:
nbac = ee.FeatureCollection("projects/sat-io/open-datasets/CA_FOREST/NBAC/nbac_1972_2023_20240530")


Lets look at the data

In [ ]:
print(nbac.size().getInfo(), nbac.first().propertyNames().getInfo())

Lets get all fires from 2023

In [9]:
nbac_2023 = nbac.filter(ee.Filter.eq('YEAR', 2023))
print(nbac_2023.size().getInfo())

2199


Let's look just at the fires in Ontario

https://cwfis.cfs.nrcan.gc.ca/downloads/nbac/NBAC_1972to2024_20250506_shp_metadata.pdf

In [10]:
nbac_on_2023 = nbac_2023.filter(ee.Filter.eq('ADMIN_AREA', 'ON'))
nbac_on_2023.size().getInfo()

89

Always a good idea to check column or property types

In [ ]:
nbac_on_2023.limit(0).getInfo()['columns']

In [11]:
ee.Number(nbac_on_2023.first().get('YEAR')).getInfo()

2023

lets visualize it

In [ ]:
centroid = nbac_on_2023.geometry().centroid(1).getInfo()['coordinates']
# geemap often automatically zooms to the layer extent when added,
# but explicitly setting a zoom level can help if needed.
# A zoom level between 8 and 10 is usually good for a provincial park.
zoom_level = 9

m = geemap.Map(center=(centroid[1], centroid[0]), zoom=zoom_level)

# Visualization parameters are not strictly needed for feature collections/geometries,
# but we can add a simple outline color if the default is hard to see.
fires_vis = {'color': 'FF0000'} # Red outline

# Add the AOI layer using aoi_ee
m.add_layer(nbac_on_2023, fires_vis, "Wildfires 2023")

# Add the layer manager to easily toggle layers
m.add_layer_manager()

m

Lets calculate some summary statistics

total area burned?

In [ ]:
# Calculate the area of each feature and add it as a property
def add_area(feature):
  return feature.set({'area_sq_ha': feature.geometry().area().divide(10000)})

nbac_on_2023 = nbac_on_2023.map(add_area)

# Calculate the total area
total_area = nbac_on_2023.aggregate_sum('area_sq_ha')

# Print the total area
print('Total area:', total_area.getInfo())


## Exploring raster datasets using NBAC feature collection


[High-Resolution Annual Forest Land Cover (1984–2022)](https://gee-community-catalog.org/projects/ca_lc/)

**Exercise:**

Calculate the % area in each fire perimeter that is coniferous.


1.   Filter image collection by date
2.   Clip image to fire perimter for each feature and
3.   *Masks all none*
4.   calculate the percent area in each class
5.   export as csv



**Class codes for raster:**

* Class Code: 0	Unclassified
* Class Code: 20	Water
* Class Code: 31	Snow/Ice
* Class Code: 32	Rock/Rubble
* Class Code: 33	Exposed/Barren Land
* Class Code: 40	Bryoids
* Class Code: 50	Shrubs
* Class Code: 80	Wetland
* Class Code: 81	Wetland Treed
* Class Code: 100	Herbs
* Class Code: 210	Coniferous
* Class Code: 220	Broad Leaf
* Class Code: 230	Mixedwood

In [ ]:
forest_lc = ee.ImageCollection("projects/sat-io/open-datasets/CA_FOREST_LC_VLCE2")
forest_lc

Lets break it down for one fire

In [ ]:
first_fire = nbac_on_2023.first()


First we get the year and make a start and end date to filter the image collection

In [ ]:
year = ee.Number(first_fire.get('YEAR')).toInt().subtract(1)
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)

get the geometry of the fire

In [ ]:
geom = first_fire.geometry()

Filter the image collection

In [ ]:
 # Filter image collection by date and bounds
image = forest_lc.filterDate(start_date, end_date).filterBounds(geom).first()

Mask all values that are not coniferous.

This is a very important aspect of getting data from raster images.

In Google Earth Engine (GEE), masking is the process of hiding or excluding certain pixels in a raster image based on specific criteria—such as pixel values, quality flags, or spatial boundaries. This is crucial for focusing analysis only on relevant data, improving accuracy, and reducing computational load.

For example, when analyzing land cover, you might mask out all pixels except those representing forests. Masking ensures that calculations like area, statistics, or classification are applied only to meaningful regions, making it a foundational step in most remote sensing workflows.

In Google Earth Engine (GEE), creating and applying a mask involves a few clear steps:

* Define the Condition: Decide which pixels you want to keep. For example, to keep only pixels with a value of 210, you use image.eq(210).
* Create the Mask: This condition returns a binary image (mask) where pixels that meet the condition are 1 (true) and others are 0 (false).
* Apply the Mask: Use .updateMask(mask) on the original image. This hides all pixels where the mask is 0, keeping only the desired ones.
* Use the Masked Image: The resulting image can now be used for further analysis, such as calculating area, statistics, or visualization.

In [ ]:
# clip image
img = ee.Image(image).clip(geom)
# create mask - all values except 210
class_210_mask = img.eq(210)
#calculate pixel area
pixel_area = ee.Image.pixelArea().divide(10000)  # hectares
# apply mask - masks all values except 210
area_image = pixel_area.updateMask(class_210_mask)

Calculate the area using the sum reducer


In [ ]:
coniferous_area = area_image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=geom,
            scale=30,
            maxPixels=1e13
        ).get('area')

lets look at the results

In [ ]:
coniferous_area

In [ ]:
first_fire = first_fire.set('class_210_area_ha', coniferous_area)

How much of the fire is coniferous?

In [ ]:
first_fire = first_fire.set('area_difference', ee.Number(first_fire.get('class_210_area_ha')).divide(ee.Number(first_fire.get('POLY_HA'))).multiply(100))
first_fire.get('area_difference').getInfo()

We can make this easier by combining everything in to one function and mapping it over the feature collection

In [ ]:
def calculate_class_210_area(feature):
    year = ee.Number(feature.get('YEAR')).toInt().subtract(1)
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    geom = feature.geometry()

    # Filter image collection by date and bounds
    image = forest_lc.filterDate(start_date, end_date).filterBounds(geom).first()

    # If image exists, process it
    def process(img):
        img = ee.Image(img).clip(geom)
        class_210_mask = img.eq(210)
        pixel_area = ee.Image.pixelArea().divide(10000)  # hectares
        area_image = pixel_area.updateMask(class_210_mask)

        area = area_image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=geom,
            scale=30,
            maxPixels=1e13
        ).get('area')

        return feature.set('class_210_area_ha', area)

    return ee.Algorithms.If(image, process(image), feature.set('class_210_area_ha', 0))




In [ ]:
#Apply to feature collection
nbac_on_2023_coniferous = nbac_on_2023.map(calculate_class_210_area)
nbac_on_2023_coniferous

## Canada Landsat-Derived Forest Harvest Disturbance (1985–2020)


[Landsat-Derived Forest Harvest Disturbance](https://gee-community-catalog.org/projects/ca_species_ts/)

**Exercise**

Were there any harvest disturbances detected within fire perimeters between 1985 and 2020?

tips:
* read in image
* clip image
* calculate max
* set as feature property
* output as csv

In [24]:
# solution

%%capture

def add_max_pixel_value(feature_collection):

  image = ee.Image("projects/sat-io/open-datasets/CA_FOREST/CA_Forest_Harvest_1985-2020")

  def calculate_max(feature):
    # Get the geometry of the feature.
    geom = feature.geometry()

    # Clip the image to the feature's geometry.
    clipped_image = image.clip(geom)

    # Calculate the maximum pixel value within the clipped image.
    max_value = clipped_image.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=geom,
        scale=image.projection().nominalScale(),
        maxPixels=1e13
    ).get(image.bandNames().get(0)) # Assuming a single band image


    # Set the max_value as a property of the feature and harvested as a property of the feature
    return feature.set('max_value', max_value)

  processed_collection = feature_collection.map(calculate_max)

  # Convert the collection to a list of dictionaries
  data_list = processed_collection.getInfo()['features']

  # Extract properties from each feature
  data_dicts = [feature['properties'] for feature in data_list]

  # Create a DataFrame
  df = pd.DataFrame(data_dicts)

  # for max value variables that are not NaN, make sure they are integers
  df['max_value'] = df['max_value'].fillna(0).astype(int)

  # max value is 0, make a new column called harvested and make it 0 but if it is greater than 0 make it 1
  df['harvested'] = df['max_value'].apply(lambda x: 1 if x > 0 else 0)


  # Map the function over the feature collection.
  return df




In [25]:
%%capture

harvest_history = add_max_pixel_value(nbac_on_2023)
harvest_history

,ADJ_FLAG,ADJ_HA,ADMIN_AREA,AG_EDATE,AG_SDATE,BASRC,CAPDATE,FIRECAUS,FIREMAPM,FIREMAPS,...,HS_EDATE,HS_SDATE,NATPARK,NFIREID,POLY_HA,PRESCRIBED,VERSION,YEAR,max_value,harvested
0,true,45.175511,ON,1.687504e+12,1.685862e+12,Agency,NaN,Human,Undefined,Undefined,...,NaN,NaN,,1639,48.380737,,20240530,2023,0,0
1,,5978.233802,ON,1.690355e+12,1.685171e+12,MAFiMS,1.690873e+12,Human,Processed imagery,Sentinel-2,...,1.686294e+12,1.685257e+12,,156,5978.233802,,20240530,2023,2020,1
2,true,93.897801,ON,1.687244e+12,1.685084e+12,Agency,NaN,Human,Undefined,Undefined,...,NaN,NaN,,1621,103.556606,,20240530,2023,2001,1
3,,1771.375233,ON,1.696230e+12,1.688195e+12,MAFiMS,1.690873e+12,Natural,Processed imagery,Landsat,...,1.687763e+12,1.687331e+12,,186,1771.375233,,20240530,2023,2015,1
4,,54033.282093,ON,1.696576e+12,1.685948e+12,MAFiMS,1.693379e+12,Natural,Processed imagery,Landsat,...,1.691996e+12,1.686553e+12,,203,54033.282093,,20240530,2023,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,,481.258343,ON,1.692947e+12,1.686294e+12,MAFiMS,1.689491e+12,Natural,Processed imagery,Sentinel-2,...,1.686899e+12,1.686553e+12,,175,481.258343,,20240530,2023,1990,1
85,,3086.398557,ON,1.692947e+12,1.686294e+12,MAFiMS,1.691478e+12,Natural,Processed imagery,Landsat,...,1.688627e+12,1.687331e+12,,174,3086.398557,,20240530,2023,2014,1
86,true,74.376855,ON,1.693984e+12,1.690873e+12,Agency,NaN,Natural,Undefined,Undefined,...,1.690960e+12,1.690960e+12,,1633,81.263931,,20240530,2023,2010,1
87,true,78.024910,ON,1.688022e+12,1.685862e+12,Agency,NaN,Natural,Undefined,Undefined,...,NaN,NaN,,1627,85.413776,,20240530,2023,0,0
